In this part we will merge hits and place them.

In [ ]:
target_name = '???'

mol_df = pd.read_pickle(f'input/{target_name}_df.p')

with open(f'input/{target_name}_reference.clean.pdb') as fh:
    pdb_block = fh.read()

## Run the next cell
No need to change settings.
But do note the `logging` module. Q: What is it?

In [ ]:
# output folder
import os
from typing import Sequence

working_folder = 'fragmenstein_data'
if not os.path.exists(working_folder):
    os.mkdir(working_folder)

# local module
from misc_funs import display_mols  # just rdkit.Chem.MolsToGridImage + IPython.display.display

# ## Start PyRosetta
# Leave alone and just run it. 
# Only one that you might want to change is `ignore_waters`
# as merging a ligand with its watershell may be a reasonable thing to do
# in extreme circumstances —like not even Chuck Norris could find a followup.

import pyrosetta, logging
import pyrosetta_help as ph

# Do not optimise hydrogen on loading:
no_optH = False  #@param {type:"boolean"}
# Ignore (True) or raise error (False) if novel residue (e.g. ligand) —  **don't tick this**.
ignore_unrecognized_res = False  #@param {type:"boolean"}
# Use autogenerated PDB residues are often weird (bad geometry, wrong match, protonated etc.): —best do it properly and parameterise it, so **don't tick this**.
load_PDB_components = False  #@param {type:"boolean"}
# Ignore all waters:
ignore_waters = True  #@param {type:"boolean"}

extra_options = ph.make_option_string(no_optH=no_optH,
                                      ex1=None,
                                      ex2=None,
                                      mute='all',
                                      ignore_unrecognized_res=ignore_unrecognized_res,
                                      load_PDB_components=load_PDB_components,
                                      ignore_waters=ignore_waters)

# capture to log
# circuitous as I needed to debug...
logger = ph.configure_logger()
logger.handlers[0].setLevel(logging.ERROR)  # logging.WARNING = 30
pyrosetta.init(extra_options=extra_options,
               )

## Optional

In the notebook, https://github.com/matteoferla/Fragmenstein/blob/master/colab_fragmenstein.ipynb,
is a section about energy minimisation of the target protein.
To increase accuracy and speed if many targets.

Q: what is a forcefield and why it the Gibbs _potential_ of folding negative?
Q: what does energy minimisation mean?

In [ ]:
template_pose: pyrosetta.Pose = pyrosetta.pose_from_file()